# Superstore Sales Analysis
The initial stage of data analysis entails pre-processing, which encompasses several steps:

Step 1: Eliminating duplicate rows, except for the Row_ID column.
 
Step 2: Removing rows that contain a few missing values.
 
Step 3: Disregarding any irrelevant values in each column, if any.
Ensuring that all values in a column are valid (e.g., ensuring that the order date and ship date values are in the correct date format). Additionally, for each entry in the dataset, the ship date must be greater than or equal to the order date.
 
Step 4: Exporting the cleaned dataset as a .csv file with a preference for UTF-8 encoding

In [1]:
import torch 
import neuralprophet
from neuralprophet import NeuralProphet, set_log_level
from neuralprophet import set_random_seed 
set_log_level("ERROR", "INFO")

import numpy as np
set_random_seed(0)
import pandas as pd
from sklearn.preprocessing import StandardScaler

import copy 

In [2]:
# Utilizing the data file "superstore_final_dataset.csv". For easy interpratation we will convert it into dataframe 
import pandas as pd
import numpy as np
df = pd.read_csv("superstore_final_dataset.csv")
df.head()
 

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold N Roll Cart System,22.3680


In [3]:
df.shape
#it has 9800 rows and 18 columns of different data type

(9800, 18)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row_ID         9800 non-null   int64  
 1   Order_ID       9800 non-null   object 
 2   Order_Date     9800 non-null   object 
 3   Ship_Date      9800 non-null   object 
 4   Ship_Mode      9800 non-null   object 
 5   Customer_ID    9800 non-null   object 
 6   Customer_Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal_Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product_ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub_Category   9800 non-null   object 
 16  Product_Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

##### So dataset ad 9800 entries with 18 different entries which are of different data type float64(2), int64(1), object(15)

### Step 1: Eliminating duplicate rows, except for the Row_ID column.

In [5]:

df.keys()
colname=['Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode',
       'Customer_ID', 'Customer_Name', 'Segment', 'Country', 'City', 'State',
       'Postal_Code', 'Region', 'Product_ID', 'Category', 'Sub_Category',
       'Product_Name', 'Sales']
df.drop_duplicates(subset=colname,inplace=True)
df.shape


(9799, 18)

### Step 2: Removing rows that contain a few missing values.


In [6]:
df.isnull().sum()

Row_ID            0
Order_ID          0
Order_Date        0
Ship_Date         0
Ship_Mode         0
Customer_ID       0
Customer_Name     0
Segment           0
Country           0
City              0
State             0
Postal_Code      11
Region            0
Product_ID        0
Category          0
Sub_Category      0
Product_Name      0
Sales             0
dtype: int64

 Null Value : only postal_code has 11 null value 

In [7]:
nan_values = df[df.isna().any(axis=1)]
#print(len(nan_values),nan_values) #= 11
null_values=df[df.isnull().any(axis=1)]
#print(len(null_values),null_values)

df.dropna(axis = 0, how ='any',inplace=True)
df.shape

(9788, 18)

### Step3 Remove irrelevant values from each column if any.

In [8]:
#Validation of all values for a column( order date and ship date value must be in correct date format )
#For each entry in dataset ship date >= order date
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'])
#df[['OD', 'SD']] = df[['Order_Date', 'Ship_Date']].apply(pd.to_datetime)#,dayfirst=True)
df['order_ship']= df['Ship_Date'] >= df['Order_Date']
df['order_ship'].value_counts()
#df.keys()
#drop newly add columns
#df.drop(columns=['OD', 'SD'],axis=1,inplace=True)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '15/04/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '22/11/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '13/05/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '17/09/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '31/01/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '17/06/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '19/11/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '28/11/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '26/08/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '28/12/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '23/04/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '30/08/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '14/04/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '22/06/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '21/08/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

...

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '19/06/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '27/02/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '16/09/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '27/02/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '31/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '31/05/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '29/01/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '19/02/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '13/07/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)

WARN

WARNING - (py.warnings._showwarnmsg) - /Users/raman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '13/05/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)



True     8104
False    1684
Name: order_ship, dtype: int64

In [9]:
df.tail

<bound method NDFrame.tail of       Row_ID        Order_ID Order_Date  Ship_Date       Ship_Mode  \
0          1  CA-2017-152156 2017-08-11 2017-11-11    Second Class   
1          2  CA-2017-152156 2017-08-11 2017-11-11    Second Class   
2          3  CA-2017-138688 2017-12-06 2017-06-16    Second Class   
3          4  US-2016-108966 2016-11-10 2016-10-18  Standard Class   
4          5  US-2016-108966 2016-11-10 2016-10-18  Standard Class   
...      ...             ...        ...        ...             ...   
9795    9796  CA-2017-125920 2017-05-21 2017-05-28  Standard Class   
9796    9797  CA-2016-128608 2016-12-01 2016-01-17  Standard Class   
9797    9798  CA-2016-128608 2016-12-01 2016-01-17  Standard Class   
9798    9799  CA-2016-128608 2016-12-01 2016-01-17  Standard Class   
9799    9800  CA-2016-128608 2016-12-01 2016-01-17  Standard Class   

     Customer_ID     Customer_Name    Segment        Country             City  \
0       CG-12520       Claire Gute   Consumer  U

In [10]:
df.keys()
#drop newly add columns
df.drop(df[df['order_ship']== False].index, inplace=True)
#df.drop(columns=['OD', 'SD','order_ship'],axis=1,inplace=True)
df.shape

(8104, 19)

After removing the values which are 1684 due to irrelevant values we need reset-index off the data fframe and remove the 3 added column.

In [11]:
df.drop(columns=['order_ship'],axis=1,inplace=True)


In [12]:
#df.reset_index()

In [13]:
df.dtypes


Row_ID                    int64
Order_ID                 object
Order_Date       datetime64[ns]
Ship_Date        datetime64[ns]
Ship_Mode                object
Customer_ID              object
Customer_Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal_Code             float64
Region                   object
Product_ID               object
Category                 object
Sub_Category             object
Product_Name             object
Sales                   float64
dtype: object

### Step4 :Export the cleaned dataset as a .csv file: prefer UTF-8 encoding


In [14]:

df.to_csv('superstore_final.csv', encoding='utf-8', index=False)

In [15]:
!tail superstore_final.csv

9779,CA-2015-169019,2015-07-26,2015-07-30,Standard Class,LF-17185,Luke Foster,Consumer,United States,San Antonio,Texas,78207.0,Central,OFF-BI-10001679,Office Supplies,Binders,GBC Instant Index System for Binding Systems,8.88
9780,CA-2015-169019,2015-07-26,2015-07-30,Standard Class,LF-17185,Luke Foster,Consumer,United States,San Antonio,Texas,78207.0,Central,OFF-AP-10003281,Office Supplies,Appliances,Acco 6 Outlet Guardian Standard Surge Suppressor,4.836
9781,CA-2017-153178,2017-09-14,2017-09-18,Standard Class,CL-12565,Clay Ludtke,Consumer,United States,Long Beach,New York,11561.0,East,TEC-PH-10001944,Technology,Phones,Wi-Ex zBoost YX540 Cellular Phone Signal Booster,437.85
9782,CA-2017-153178,2017-09-14,2017-09-18,Standard Class,CL-12565,Clay Ludtke,Consumer,United States,Long Beach,New York,11561.0,East,OFF-BI-10004390,Office Supplies,Binders,GBC DocuBind 200 Manual Binding Machine,673.568
9787,US-2015-114377,2015-05-11,2015-05-11,Same Day,BG-11035,Barry Gonzalez,Consumer,United State

In [16]:
import pandas as pd
import sqlite3


In [17]:
df = pd.read_csv('superstore.csv', encoding='utf-8')

In [18]:
df

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940
2,13,CA-2018-114412,15/04/2018,20/04/2018,Standard Class,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,28027.0,South,OFF-PA-10002365,Office Supplies,Paper,Xerox 1967,15.552
3,14,CA-2017-161389,05/12/2017,10/12/2017,Standard Class,IM-15070,Irene Maddox,Consumer,United States,Seattle,Washington,98103.0,West,OFF-BI-10003656,Office Supplies,Binders,Fellowes PB200 Plastic Comb Binding Machine,407.976
4,15,US-2016-118983,22/11/2016,26/11/2016,Standard Class,HP-14815,Harold Pawlan,Home Office,United States,Fort Worth,Texas,76106.0,Central,OFF-AP-10002311,Office Supplies,Appliances,Holmes Replacement Filter for HEPA Air Cleaner...,68.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8099,9792,CA-2015-127166,21/05/2015,23/05/2015,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Houston,Texas,77070.0,Central,OFF-EN-10003134,Office Supplies,Envelopes,Staple envelope,56.064
8100,9793,CA-2015-127166,21/05/2015,23/05/2015,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Houston,Texas,77070.0,Central,FUR-CH-10003396,Furniture,Chairs,Global Deluxe Steno Chair,107.772
8101,9794,CA-2015-127166,21/05/2015,23/05/2015,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Houston,Texas,77070.0,Central,OFF-PA-10001560,Office Supplies,Paper,"Adams Telephone Message Books, 5 1/4” x 11”",4.832
8102,9795,CA-2015-127166,21/05/2015,23/05/2015,Second Class,KH-16360,Katherine Hughes,Consumer,United States,Houston,Texas,77070.0,Central,OFF-BI-10000977,Office Supplies,Binders,Ibico Plastic Spiral Binding Combs,18.240


In [19]:
conn = sqlite3.connect("superstore.sql")

In [20]:
df.to_sql('Some_Table_Name', onn)

NameError: name 'onn' is not defined

In [ ]:
df.to_sql('superstore', onn)

In [21]:
sql_string = 'SELECT * FROM superstore'

CSV to sql

convert the date time to datetime format

1. Write an SQL query to solve the given problem statement.
What percentage of total orders were shipped on the same date?


In [22]:
df['same_day'] =df['Order_Date']==df['Ship_Date']
df['same_day'].value_counts()
df['same_day'].value_counts()*100/df.shape[0]

False    93.657453
True      6.342547
Name: same_day, dtype: float64

2. Write an SQL query to solve the given problem statement. Name top 3 customers with highest total value of orders.


In [23]:
#df['Customer_Name'].value_counts()[:3]

In [24]:
df.groupby(['Customer_Name'])['Sales'].sum().sort_values(ascending=False)[:3]#.index.tolist()

Customer_Name
Sean Miller     24516.600
Tamara Chand    19044.906
Raymond Buch    15056.854
Name: Sales, dtype: float64

3. Write an SQL query to solve the given problem statement. Find the top 5 items with the highest average sales per day.


In [25]:
#Product_Name , sales, date
#df.groupby(['Product_ID','Order_Date'])['Sales'].mean()
#df.groupby(['Product_ID','Order_Date']).sum().groupby('Product_ID')['Sales'].mean().sort_values(ascending=False)[:5]

In [26]:
df.groupby('Product_ID')['Sales'].mean().sort_values(ascending=False)[:5]

Product_ID
TEC-MA-10002412    22638.4800
TEC-CO-10004722    12319.9648
TEC-MA-10004125     7999.9800
TEC-MA-10001047     7149.9450
TEC-MA-10001127     6124.9650
Name: Sales, dtype: float64

#sql
SELECT Product_ID, AVG(sales) AS Average_sales
FROM superstore
GROUP BY Product_ID
ORDER BY  Average_sales DESC
LIMIT 5;


4. Write a query to find the average order value for each customer, and rank the customers by their average order value.

In [27]:
df.groupby('Customer_Name')['Sales'].mean().sort_values(ascending=False)

Customer_Name
Tom Ashbrook         2301.295000
Mitch Willingham     1751.292000
Sean Miller          1751.185714
Tamara Chand         1731.355091
Grant Thornton       1558.535333
                        ...     
Ritsa Hightower         8.226000
Roy Skaria              7.442667
Michelle Lonsdale       6.370000
Lela Donovan            5.304000
Thais Sissman           2.025000
Name: Sales, Length: 791, dtype: float64

5. Write a query to find Give the name of customers who ordered highest and lowest orders from each city.

In [28]:
df.groupby(['City'])['Sales'].max()

City
Aberdeen         25.500
Abilene           1.392
Akron           665.408
Albuquerque     288.240
Alexandria     4164.050
                 ...   
Woonsocket       52.960
Yonkers        4899.930
York            422.058
Yucaipa          50.800
Yuma            599.985
Name: Sales, Length: 506, dtype: float64

In [29]:
df_new = df[['City', 'Sales','Customer_Name']]
df_new

,City,Sales,Customer_Name
0,Henderson,261.960,Claire Gute
1,Henderson,731.940,Claire Gute
2,Concord,15.552,Andrew Allen
3,Seattle,407.976,Irene Maddox
4,Fort Worth,68.810,Harold Pawlan
...,...,...,...
8099,Houston,56.064,Katherine Hughes
8100,Houston,107.772,Katherine Hughes
8101,Houston,4.832,Katherine Hughes
8102,Houston,18.240,Katherine Hughes


In [30]:
#City, Customer_Name, sales
idmax = df_new.groupby(['City'])['Sales'].idxmax()

In [31]:
idmin=df_new.groupby(['City'])['Sales'].idxmin()

In [32]:
df_new.loc[idmax]
df_new.loc[idmin]
#result = pd.concat([df_new.loc[idmax],df_new.loc[idmin] ], axis=1, join="outr")

,City,Sales,Customer_Name
7659,Aberdeen,25.500,Jeremy Lonsdale
5815,Abilene,1.392,Dennis Kane
3558,Akron,2.286,Karl Braun
6729,Albuquerque,4.280,Susan Vittorini
2094,Alexandria,12.420,Shirley Daniels
...,...,...,...
4210,Woonsocket,51.450,Kristen Hastings
3161,Yonkers,10.896,Nathan Cano
1720,York,38.088,Theresa Swint
2735,Yucaipa,50.800,Odella Nelson


In [33]:
df_new.loc[idmax]

,City,Sales,Customer_Name
7659,Aberdeen,25.500,Jeremy Lonsdale
5815,Abilene,1.392,Dennis Kane
355,Akron,665.408,Ed Braxton
2504,Albuquerque,288.240,Benjamin Farhat
2681,Alexandria,4164.050,Greg Maxwell
...,...,...,...
3269,Woonsocket,52.960,Andy Reiter
860,Yonkers,4899.930,Karen Daniels
1719,York,422.058,Theresa Swint
2735,Yucaipa,50.800,Odella Nelson


 6. Write an SQL query to solve What is the most demanded sub-category in the west region?
 


In [34]:
df[df['Region']=='West'].groupby(['Sub_Category'])['Sales'].sum().sort_values(ascending=False)[:1]

Sub_Category
Chairs    86795.76
Name: Sales, dtype: float64

7. Write an SQL query to solve Which order has the highest number of items? And which order has the highest cumulative value?


In [35]:
df.groupby(['Order_ID'])['Sub_Category'].count().sort_values(ascending=False)[:1]

Order_ID
CA-2018-100111    14
Name: Sub_Category, dtype: int64

SELECT Order_ID, COUNT(Sub_Category) AS highest_number
FROM superstore
GROUP BY Order_ID
ORDER BY  highest_number DESC
LIMIT 1;